In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import random
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym_super_mario_bros.actions import COMPLEX_MOVEMENT


from tensorflow.python.framework.ops import disable_eager_execution

In [3]:
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
height, width, channels = env.observation_space.shape

action_space = env.get_action_meanings()  # 
actions = env.action_space.n #does not work cuz the game usese joypad

In [4]:
print(actions)

7


In [5]:
env.unwrapped.get_action_meanings()

['NOOP']

In [6]:
action_space

['NOOP', 'right', 'right A', 'right B', 'right A B', 'A', 'left']

In [8]:
'''episode = 5
for episode in range(1, episode+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5,6])
        n_state, reward, done, info = env.step(action)
        #env.step(valid_actions[np.random.randint(3)]) 
        #env.step(action_space)
        score+=reward
    print('Episode:{} socre:{}'.format(episode, score))

env.close()'''

"episode = 5\nfor episode in range(1, episode+1):\n    state = env.reset()\n    done = False\n    score = 0\n    \n    while not done:\n        env.render()\n        action = random.choice([0,1,2,3,4,5,6])\n        n_state, reward, done, info = env.step(action)\n        #env.step(valid_actions[np.random.randint(3)]) \n        #env.step(action_space)\n        score+=reward\n    print('Episode:{} socre:{}'.format(episode, score))\n\nenv.close()"

# Creating a Deep Learning Model with keras

In [4]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(128, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (4,4), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model    

In [5]:
model = build_model(height, width, channels, actions)

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 59, 63, 128)    24704     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 28, 30, 64)     131136    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 25, 27, 64)     65600     
_________________________________________________________________
flatten (Flatten)            (None, 129600)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               66355712  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 1

# Build an Agent with keras

In [7]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [27]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, 
                   memory=memory, 
                   policy=policy,
                   enable_dueling_network=True, 
                   dueling_type='avg', 
                   nb_actions=actions, 
                   nb_steps_warmup=1000,
                  )
    return dqn

In [20]:
disable_eager_execution()

In [ ]:
#actionss = np.array(action_space)
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3))

In [ ]:
env.render()
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)